In [ ]:
!pip install geopandas shapely

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 4.9 MB/s 
     |████████████████████████████████| 6.3 MB 32.4 MB/s 
     |████████████████████████████████| 16.7 MB 218 kB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import ee
import pandas as pd
import geopandas as gpd

from shapely.geometry.multipolygon import MultiPolygon
from shapely.geometry.polygon import Polygon

There are two types of dataset needed to estimate the Foliar and Ground cover variables: 1- The time series of Landsat images, 2- The time series of PRISM climatic data. Also, we need the DEM data for estimating the slope and slope shape of each location.

In this notebook we will provide the code to download all of them from the Google Earth Engine service.

Authentication for Earth Engine: we need to connect to the earth engine account and send the requests from the notebook. Please set up your ee account in advance.

In [ ]:
ee.Authenticate()
ee.Initialize()

### 1- Get the time series of Landsat

We read the polygon shapefile of the area that we are going to download the data for. 

**Any shapefile that you are going to use should have these two columns: 'PrimaryKey', 'geometry'**

If each one of these columns does not exist you may edit the data and create it

In [ ]:
buffs = gpd.read_file('/content/drive/MyDrive/RHEM_mapping/Vectors/Area.shp').loc[:,['PK_EE', 'geometry']]
buffs['PrimaryKey'] = buffs['PK_EE'] ### We create the PrimaryKey column by another column named "PK_EE"
buffs

,PK_EE,geometry,PrimaryKey
0,1,"POLYGON ((-470965.477 251121.920, -470969.307 ...",1
1,2,"POLYGON ((-470917.138 281370.940, -470916.923 ...",2


Define the Coordinate system that you would like to get the data in that form. It should be defined in WKT format. You may use <a href="https://epsg.io/">epsg website</a> to get the WKT format of a coordinate system. Here, we used "USA_Contiguous_Lambert_Conformal_Conic" (EPSG:102004):

In [ ]:
proj = ee.Projection('\
    PROJCS["USA_Contiguous_Lambert_Conformal_Conic", \
    GEOGCS["GCS_North_American_1983",\
        DATUM["North_American_Datum_1983",\
            SPHEROID["GRS_1980",6378137,298.257222101]],\
        PRIMEM["Greenwich",0],\
        UNIT["Degree",0.017453292519943295]],\
    PROJECTION["Lambert_Conformal_Conic_2SP"],\
    PARAMETER["False_Easting",0],\
    PARAMETER["False_Northing",0],\
    PARAMETER["Central_Meridian",-96],\
    PARAMETER["Standard_Parallel_1",33],\
    PARAMETER["Standard_Parallel_2",45],\
    PARAMETER["Latitude_Of_Origin",39],\
    UNIT["Meter",1],\
    AUTHORITY["EPSG","102004"]]')

Create the <a href="https://developers.google.com/earth-engine/guides/ic_creating">image collection</a> by stacking <a href="https://developers.google.com/earth-engine/datasets/catalog/landsat">Landsat4 to 8 repositories</a>. We also need to rename the bands of Landsat8 data to match them with Landsat4 to 7:

In [ ]:
listOfSensors = [
                "LANDSAT/LT04/C01/T1_SR",
                "LANDSAT/LT05/C01/T1_SR",
                "LANDSAT/LE07/C01/T1_SR",
                "LANDSAT/LC08/C01/T1_SR"
                ]

collectAll = []
for i in range(4):
    collect0 = ee.ImageCollection(listOfSensors[i])
    if i == 3:
        qual= 'IMAGE_QUALITY_TIRS'
        collect0 = collect0.filterMetadata(qual, 'equals', 9)
        collect0 = collect0.select(['B2','B3','B4','B5','B6', 'B7', 'pixel_qa'], ['B1','B2','B3','B4','B5', 'B7', 'pixel_qa'])
        collectAll.append(collect0)
    else:
        qual = 'IMAGE_QUALITY'
        collect0 = collect0.filterMetadata(qual, 'equals', 9)
        collect0 = collect0.select(['B1','B2','B3','B4','B5', 'B7', 'pixel_qa'])
        collectAll.append(collect0)

collect0 = collectAll[0].merge(collectAll[1]).merge(collectAll[2]).merge(collectAll[3])

Next, we send the download request to the Earth Engine service for each polygon of the shapefile. Before running the cell below, Within the **main directory** of your Google Drive, create a folder for each polygon of the shapefile and named them by their PrimaryKey values. Here we have two polygons with PrimaryKey values of 1 and 2.

In [ ]:
from ee import feature
for i in range(len(buffs.index.tolist())):
    
    # select the polygon
    point = buffs.iloc[[i]]
    folder_name = str(point['PrimaryKey'].values[0])

    # filter collection by the polygons's bounds
    polygonBounds = point.total_bounds.tolist()

    # filter the image collection by the polygons's bounds
    polygon = ee.Geometry.Rectangle([polygonBounds[0],polygonBounds[1],polygonBounds[2],
                                         polygonBounds[3]], proj=proj, evenOdd=False)
    collectSR = collect0.filterBounds(polygon)

    # filter collection by the time range of interest. Here we would like to get 20 seasons before Summer 2020
    first_date = '2015-09-01'
    last_date = '2020-09-01'
    collectSR = collectSR.filterDate(first_date, last_date).sort('system:time_start')

    print('There are', collectSR.size().getInfo(), 'Landsat images between', first_date, 'and', last_date, 'around polygon '+folder_name)


    if collectSR.size().getInfo() != 0:

      listOfDate = ee.List(collectSR.aggregate_array('system:time_start'))
      listOfDate = listOfDate.map(lambda time_start: ee.Date(time_start).format('Y-MM-dd'))

      listOfID = ee.List(collectSR.aggregate_array('system:id'))
      listOfSAT = ee.List(collectSR.aggregate_array('SATELLITE'))

      feature_list = []
      for t in range(collectSR.size().getInfo()):
        feature_list.append(ee.Feature(None, ee.Dictionary({'Unnamed: 0': t+1,'date': listOfDate.get(t), 'Image_ID': listOfID.get(t), 'Satellite': listOfSAT.get(t)})))
      datesDF_SR = ee.FeatureCollection(feature_list)

      taskToexport = ee.batch.Export.table.toDrive(
                                                  collection= datesDF_SR,
                                                  description= 'dates_SR',
                                                  fileFormat= 'CSV',
                                                  folder = folder_name,
                                                  selectors = ['Unnamed: 0', 'date', 'Image_ID', 'Satellite']
                                                  )

      taskToexport.start()
      
      # list of Landsat bands to iterate
      listOfBands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa']

      # iterate over the bands and export to GDrive folder
      for bands in listOfBands:
          band1 = collectSR.select([bands]).toBands()
          taskToexport = ee.batch.Export.image.toDrive(
              image = band1,
              crs = proj.getInfo()['wkt'],             
              region = polygon,
              description = bands,
              scale = 30,
              folder = folder_name
              )

          taskToexport.start()
      
      print('Request sent for', folder_name)
    
    else:
      print('No image exists for {}. Please check the region and date range'.format(folder_name))


There are 540 Landsat images between 2015-09-01 and 2020-09-01 around polygon 1
Request sent for 1
There are 549 Landsat images between 2015-09-01 and 2020-09-01 around polygon 2
Request sent for 2


### 2- Get the time series of PRISM

We read the polygon shapefile of the area that we are going to download the data for. 

**Any shapefile that you are going to use should have these two columns: 'PrimaryKey', 'geometry'**

If each one of these columns does not exist you may edit the data and create it.

Here, we only have one polygon and having unique PrimaryKey column is not necessary. But, if there are more than one polygon in the shapefile, the PrimaryKey needs to be defined because the download loop uses it.

In [ ]:
buffs = gpd.read_file('/content/drive/MyDrive/RHEM_mapping/Vectors/PRISM_EE.shp').loc[:,['PK_EE', 'geometry']]
buffs['PrimaryKey'] = buffs['PK_EE']
buffs

,PK_EE,geometry,PrimaryKey
0,1,"POLYGON ((-470917.138 281370.940, -438422.714 ...",1


Like the Landsat data, we create the <a href="https://developers.google.com/earth-engine/guides/ic_creating">image collection</a> from <a href="https://developers.google.com/earth-engine/datasets/catalog/OREGONSTATE_PRISM_AN81m">PRISM monthly data repository</a>.

We also use the same coordinate system as Landsat data for PRISM.

In [ ]:
collect0 = ee.ImageCollection('OREGONSTATE/PRISM/AN81m')

Next, we send the download request to the Earth Engine service for each polygon of the shapefile. Before running the cell below, Within the **main directory** of your Google Drive, create a folder for each polygon of the shapefile and named them by their PrimaryKey values. Here we have one polygon with PrimaryKey values of 1.

In [ ]:
for i in range(len(buffs.index.tolist())):
    
    # select the polygon
    point = buffs.iloc[[i]]
    folder_name = str(point['PrimaryKey'].values[0])

    # filter collection by the polygons's bounds
    polygonBounds = point.total_bounds.tolist()

    # filter the image collection by the polygons's bounds
    polygon = ee.Geometry.Rectangle([polygonBounds[0],polygonBounds[1],polygonBounds[2],
                                         polygonBounds[3]], proj=proj, evenOdd=False)
    collectSR = collect0.filterBounds(polygon)

    # filter collection by the time range of interest. Here we would like to get 20 seasons before Summer 2020
    first_date = '2015-09-01'
    last_date = '2020-09-01'
    collectSR = collectSR.filterDate(first_date, last_date).sort('system:time_start')

    if collectSR.size().getInfo() != 0:

      # list of PRISM bands to iterate
      listOfBands = ['ppt', 'tmean']

      # iterate over the bands and export to GDrive folder
      for bands in listOfBands:
          band1 = collectSR.select([bands]).toBands()
          taskToexport = ee.batch.Export.image.toDrive(
              image = band1,
              crs = proj.getInfo()['wkt'],             
              region = polygon,
              description = bands,
              scale = collect0.first().select('ppt').projection().nominalScale().getInfo(), # we use the original pixel size of the PRISM data which is about 4638 m
              folder = folder_name
              )

          taskToexport.start()
      
      print('Request sent for', folder_name)
    
    else:
      print('No image exists for {}. Please check the region and date range'.format(folder_name))


Request sent for 1


### 3- Get the Elevation data

You may import the shapefile of the area that you are going to get the elevation data for. Here, we are going to use the same area as PRISM shapefile.

This time, we create the <a href="https://developers.google.com/earth-engine/guides/image_overview">image</a> from <a href="https://developers.google.com/earth-engine/datasets/catalog/WWF_HydroSHEDS_03CONDEM">HydroSHEDS data repository</a> because it only has one band which is elevation.

We also use the same coordinate system as Landsat and PRISM data.

In [ ]:
collect0 = ee.Image('WWF/HydroSHEDS/03CONDEM')

Next, we send the download request to the Earth Engine service for each polygon of the shapefile. Before running the cell below, Within the **main directory** of your Google Drive, create a folder for each polygon of the shapefile and named them by their PrimaryKey values. Here we have one polygon with PrimaryKey values of 1.

In [ ]:
for i in range(len(buffs.index.tolist())):
    
    # select the polygon
    point = buffs.iloc[[i]]
    folder_name = str(point['PrimaryKey'].values[0])

    # list of PRISM bands to iterate
    listOfBands = ['b1']

    # iterate over the bands and export to GDrive folder
    for bands in listOfBands:
        band1 = collect0.select([bands])
        taskToexport = ee.batch.Export.image.toDrive(
            image = band1,
            crs = proj.getInfo()['wkt'],             
            region = polygon,
            description = 'Elevation',
            scale = collect0.select('b1').projection().nominalScale().getInfo(), # we use the original pixel size of the HydroSHEDS data which is about 92 m
            folder = folder_name
            )

        taskToexport.start()
    
    print('Request sent for', folder_name)


Request sent for 1
